# Process Tetris Videos

Tetris videos are found in the Data folder in each user's session file.  Their names include the timestamp of their initial recording.

This Script includes two processing steps: you can pass in the participant and session ('P1', 'sess1') and it will run through each frame of the video, OCR relevant sections looking for text (i.e. 'RESUME', 'PAUSED', 'START', 'GAME OVER'), and write a metadata file that allows us to just have a simple dataframe that captures when the user is playing and when they succumb to the game.

We might develop a prior of focus state based on this game play structure.



In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import easyocr
import pandas as pd
from IPython.display import display, clear_output
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

%matplotlib inline

In [2]:
class VideoAnnotator:
    def __init__(self, folder, user, sess):
        
        full_path = os.path.join(folder + '/' + user, user + '_' + sess)
        # Find video file and extract timestamp
        for file_name in os.listdir(full_path):
            if file_name.endswith(".MP4"):
                video_file = os.path.join(full_path, file_name)
                # Assuming timestamp is the number after "Final" in the filename
                timestamp_match = re.search(r"Final(\d+)", file_name)
                if timestamp_match:
                    self.video_timestamp = int(timestamp_match.group(1))*1000
                break
        self.status = user + ', ' + sess
        self.cap = cv2.VideoCapture(video_file)
        self.reader = easyocr.Reader(['en'],gpu = False)
        self.df = pd.DataFrame(columns=['timestamp', 'status', 'ocr_string'])
    
    def progress(self, count, total, status=''):
        bar_len = 60
        filled_len = int(round(bar_len * count / float(total)))
        bar = '=' * filled_len + '-' * (bar_len - filled_len)
        clear_output(wait=True)
        display('{} [{}] {}'.format(status, bar, count))
        
    def run(self, speed=1):
        
        prev_ocr = ''
        prev_ocr_gover = ''
        
        #ROIs where we see pause/resume status and game over text
        x1, y1, x2, y2 = 750, 600, 1200, 750 
        x3, y3, x4, y4 = 725, 900, 1225, 1150
        
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_count = 0
        
        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            if not ret:
                break
        
            if frame_count % speed == 0:
            
                # Extract the ROI from the image
                roi = frame[y1:y2, x1:x2]
                roi_gameover = frame[y3:y4, x3:x4]

                #OCR it
                ocr_output = self.reader.readtext(roi)
                if len(ocr_output):
                    ocr_result = ocr_output[0][1].lower()
                else:
                    ocr_result = ''

                ocr_output_gover = self.reader.readtext(roi_gameover)
                if len(ocr_output_gover):
                    ocr_result_gover = ocr_output_gover[0][1].lower()
                else:
                    ocr_result_gover = ''

                #check if OCR has changed, indicating an update in game status
                if (prev_ocr != ocr_result or prev_ocr_gover != ocr_result_gover):

                    prev_ocr = ocr_result
                    prev_ocr_gover = ocr_result_gover

                    status = 'NOTIFICATION'

                    if (ocr_result == '' and ocr_result_gover == ''):
                        status='PLAYING'
                    elif('game' in ocr_result_gover or 'over' in ocr_result_gover):
                        status='GAME_OVER'
                    elif('start' in ocr_result or 'resume' in ocr_result):
                        status='PAUSED'
                    elif('falling' in ocr_result or 'blocks' in ocr_result or 'player' in ocr_result_gover):
                        status='BOOT_MENU'
                    elif('latest' in ocr_result or 'public' in ocr_result or 'score' in ocr_result):
                        status='SCOREBOARD'
                    elif(ocr_result != ''):
                        status='UNKNOWN'

                    new_row = {
                    'timestamp': self.cap.get(cv2.CAP_PROP_POS_MSEC) + self.video_timestamp,
                    'status': status,          # An example status
                    'ocr_string': ocr_result + ' | ' + ocr_result_gover
                    }

                    self.df = self.df.append(new_row, ignore_index=True)
            self.progress(frame_count, total_frames, status='Processing video ' + self.status)
            frame_count += 1

        self.cap.release()
        
        return self.df

## Single File Process and Save

this will overwrite existing `video_meta.pkl` in the session folder

In [1]:
folder = 'Data'
user = 9
session = 1

va = VideoAnnotator(folder, 'P' + str(user), 'sess' + str(session))
video_metadata = va.run(speed=8)
video_metadata.to_pickle(folder + '/P' + str(user) + '/P' + str(user) + '_sess' + str(session) + '/video_meta.pkl')

## Process All Videos and Save

this will *not* overwrite existing `video_meta.pkl` in the session folder.  If one exists and is incomplete, you must delete it or it will skip it.  Also skips over nonexistant participants/folders.

In [4]:
folder = 'Data'

for user in range(0,25):
    for session in [1,2]:
        print('Moving to User ' + str(user) + ', session ' + str(session))
        
        #check if file exists already
        if(os.path.exists(folder + '/P' + str(user) + '/P' + str(user) + '_sess' + str(session) + '/video_meta.pkl')):
            print('already found metadata file, moving to next...')
        
        else:
            try:
                va = VideoAnnotator(folder, 'P' + str(user), 'sess' + str(session))
                video_metadata = va.run(speed=8)
                video_metadata.to_pickle(folder + '/P' + str(user) + '/P' + str(user) + '_sess' + str(session) + '/video_meta.pkl')
            except Exception as e:
                print('ERROR! ' + str(e))

'Processing video P15, sess2 [============================================================] 161102'

Moving to User 16, session 1
ERROR! [Errno 2] No such file or directory: 'Data/P16/P16_sess1'
Moving to User 16, session 2
ERROR! [Errno 2] No such file or directory: 'Data/P16/P16_sess2'
Moving to User 17, session 1
ERROR! [Errno 2] No such file or directory: 'Data/P17/P17_sess1'
Moving to User 17, session 2
ERROR! [Errno 2] No such file or directory: 'Data/P17/P17_sess2'
Moving to User 18, session 1
ERROR! [Errno 2] No such file or directory: 'Data/P18/P18_sess1'
Moving to User 18, session 2
ERROR! [Errno 2] No such file or directory: 'Data/P18/P18_sess2'
Moving to User 19, session 1
ERROR! [Errno 2] No such file or directory: 'Data/P19/P19_sess1'
Moving to User 19, session 2
ERROR! [Errno 2] No such file or directory: 'Data/P19/P19_sess2'
Moving to User 20, session 1
ERROR! [Errno 2] No such file or directory: 'Data/P20/P20_sess1'
Moving to User 20, session 2
ERROR! [Errno 2] No such file or directory: 'Data/P20/P20_sess2'
Moving to User 21, session 1
ERROR! [Errno 2] No such file o

## Read DF Back

In [6]:
folder = 'Data'
user = 15
session = 2

metadata_read = pd.read_pickle(folder + '/P' + str(user) + '/P' + str(user) + '_sess' + str(session) + '/video_meta.pkl')
metadata_read.head(50)

,timestamp,status,ocr_string
0,1682190516410.0,PAUSED,resume |
1,1682191415565.0,PAUSED,resume | 1
2,1682191415698.333252,PAUSED,resume | l
3,1682191415831.666748,PAUSED,resume |
4,1682191415965.0,UNKNOWN,4ong 43 ou cam |
5,1682191416098.333252,UNKNOWN,as you can: |
6,1682191416231.666748,UNKNOWN,2 | 00
7,1682191416366.666748,UNKNOWN,ony 4> 9ou cam |
8,1682191416500.0,PAUSED,resume |
9,1682191417833.333252,PLAYING,|
